**Основные цели этого задания:**
*   Попрактиковаться в борьбе с дисбалансом классов
*   Научиться заполнять пропуски в данных
*   Научиться использовать категориальные признаки.

**Задача: по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята.**

*1. Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.*

*2. Подготовьте данные к обучению моделей:*
*   Отделите целевую переменную Grant.Status и выясните, сбалансированы ли классы;
*   Заполните пропуски: в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта); в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;
*   Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;
*   Разделите данные на обучающую и тестовую части;
*   Используйте масштабирование для получения признаков одинакового масштаба.

*3. Обучите модели и выберите лучшую:*
*   Обучите модель логистической регрессии, используя кросс-валидацию. Оцените ее качество с помощью метрики rocauc. Выведите топ-10 признаков по важности, согласно обученной модели;
*   Обучите модель случайного леса:
*   а) Для подбора гиперпараметров и кросс-валидации используйте структуру GridSearchCV,
*   б) Выберите наилучший вариант случайного леса и выведите его параметры,
*   в) Оцените качество выбранной модели с помощью метрики rocauc,
*   г) Выведите топ-10 признаков по важности. Используйте атрибут feature_importances_, чтобы узнать важность признаков в деревянных моделях.

*4. Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи.*